Bibliotecas


In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import math
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity


Lendo dataframes


In [55]:
dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])
filmesId=pd.DataFrame((set(dataOriginal['item id'])),columns=['Filmes'])

itemData=pd.read_csv('./ml-100k/u.item',sep=r'|',names=['movie_id ', 'movie title' , 'release date' ,' video release date' ,
              'IMDb URL' ,' unknown' , 'Action' , 'Adventure' , 'Animation' ,
             ' Childrens ', 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western' ], encoding='latin-1')
userData=pd.read_csv('./ml-100k/u.user',sep='|',names=['user id','age','Genero','Ocupacao','ZipCode'])

C:\Users\User\AppData\Local\Temp\ipykernel_26836\1420820392.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])


In [56]:
def h(rat,user,mediaUser):
    val=rat - mediaUser[user-1]
    return val

In [57]:
# def cossine_similarity2(u, v,tabelaSimilaridade,id_u,id_v):
#     itemsComuns = np.where(((u!=0) & (v!=0)))[0]
#     u=u[itemsComuns]
#     v=v[itemsComuns]
#     if(u.size==0 or v.size==0):
#         return 0
#     result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))

#     tabelaSimilaridade[id_u-1][id_v-1]=result
#     tabelaSimilaridade[id_v-1][id_u-1]=result
#     return result

In [58]:
def  cossine_Similarity(u, v,tabelaSimilaridade,id_u,id_v):
    itemsComuns = np.where(((u!=0) & (v!=0)))[0]
    j=u[itemsComuns]
    k=v[itemsComuns]
    x=u[u!=0]
    y=v[v!=0]
    if(u.size==0 or v.size==0):
        display('Não tem itens em comum')
        return 0
    result = np.dot(j, k)/(np.linalg.norm(x)*np.linalg.norm(y))

    tabelaSimilaridade[id_u-1][id_v-1]=result
    tabelaSimilaridade[id_v-1][id_u-1]=result
    return result

In [59]:
def  pearson(u, v,tabelaSimilaridade,id_u,id_v):
    itemsComuns = np.where(((u!=0) & (v!=0)))[0]
    j=u[itemsComuns]
    k=v[itemsComuns]
    x=u[u!=0]
    y=v[v!=0]
    mediaU=np.mean(u)
    mediaV=np.mean(v)
    t=j-mediaU
    z=k-mediaV
    l=x-mediaU
    m=y-mediaV
    if(u.size==0 or v.size==0):
        display('Não tem itens em comum')
        return 0
    result = np.dot(t, z)/(np.linalg.norm(l)*np.linalg.norm(m))

    tabelaSimilaridade[id_u-1][id_v-1]=result
    tabelaSimilaridade[id_v-1][id_u-1]=result
    return result

In [60]:
def user_based(user_id, item_id,data,tabelaSimilaridade,mediaUser,k,similarityFunction):
        values_to_match = np.unique(data[:, item_id-1 ])
        values_to_match = values_to_match[values_to_match != 0]
        atching_indices = np.where(np.isin(data[:, item_id-1], values_to_match))[0]
        atching_indices+=1
        Nui = pd.DataFrame(userData[userData['user id'].isin(atching_indices)]['user id'].values,columns=['user id']).reset_index(drop=True)
        if(len(Nui)<=0):
            
            display('Não tem usuários para recomendar')
            return mediaUser[user_id-1]
        if similarityFunction=='cossine':
            Nui['similaridade']=Nui['user id'].apply(lambda x:(
                                cossine_Similarity(data[user_id-1][:],
                                                    data[x-1][:],tabelaSimilaridade,user_id,x
                                                    )
                                                    if math.isnan(tabelaSimilaridade[user_id-1][x-1])else tabelaSimilaridade[user_id-1][x-1]
                                                    ))
        elif similarityFunction=='pearson':
            Nui['similaridade']=Nui['user id'].apply(lambda x:(
                                pearson(data[user_id-1][:],
                                        data[x-1][:],tabelaSimilaridade,user_id,x
                                        )
                                        if math.isnan(tabelaSimilaridade[user_id-1][x-1])else tabelaSimilaridade[user_id-1][x-1]))
        Nui=Nui[Nui['similaridade']!=0]
        Nui=Nui.sort_values(by='similaridade',ascending=False).head(k)
   
        parcial1=sum(Nui.apply(lambda x:
                                (x['similaridade']*h(data[int(x['user id'])-1][item_id-1],int(x['user id']),mediaUser)),axis=1
                                ))
        parcial2 = sum(Nui.apply(lambda x:
                                (abs(x['similaridade']) ),axis=1
                                ))
        val=mediaUser[user_id-1]
        resp=(parcial1/parcial2)+val
        # print(resp)    
        return resp

In [61]:
userItem=dataOriginal.pivot(index='user id',columns='item id',values='rating').fillna(0)
userItem[:]=0
display(userItem)
userItem=userItem.to_numpy()
userItem

item id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [62]:
# mediaUser=dataOriginal.drop(['timestamp','item id'],axis=1).groupby('user id').mean().reset_index()
# mediaUser=mediaUser.to_numpy()
# mediaUser

In [63]:
#iniciando
similaridade = pd.DataFrame(index=userData['user id'],columns=userData['user id'])
for id in userData['user id']:
    similaridade[id][id]=1
display(similaridade)
similaridade=similaridade.to_numpy()
similaridade

user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user id,,,,,,,,,,,,,,,,,,,,,
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


array([[1, nan, nan, ..., nan, nan, nan],
       [nan, 1, nan, ..., nan, nan, nan],
       [nan, nan, 1, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., 1, nan, nan],
       [nan, nan, nan, ..., nan, 1, nan],
       [nan, nan, nan, ..., nan, nan, 1]], dtype=object)

In [64]:
class MemoryBased():

    def __init__(self, tabelaSimilaridade,userItem,k=5,metrica='user_based',calculaSimilaridade='cossine'):
        self.tabelaSimilaridade = tabelaSimilaridade
        self.userItem=userItem
        self.k=k
        self.metrica=metrica
        self.calculaSimilaridade=calculaSimilaridade

    def fit(self, x_train, y_train):
        x_train['rating']=y_train
        for index, row in x_train.iterrows():
            user_id=row['user id']
            item_id=row['item id']
            self.userItem[user_id-1][item_id-1]=row['rating']
        
        # for  index in range(0,len(self.tabelaSimilaridade)):
        #     for index2  in range(index,len(self.tabelaSimilaridade)):
        #         if(index!=index2 or np.isnan(self.tabelaSimilaridade[index][index2])):	
        #             if(self.calculaSimilaridade=='cosine'):
        #                 cossine_Similarity(self.userItem[index],self.userItem[index2],self.tabelaSimilaridade,index,index2)
                    
        return self.userItem
    def predict(self, x):
        y_pred = []
        mediaUser=self.userItem.mean(axis=1)
        display(mediaUser[942])
        for index, row in x.iterrows():
            user_id=row['user id']
            item_id=row['item id']
            if(self.metrica=='user_based'):
                if self.calculaSimilaridade=='cossine':
                    temp=user_based(user_id,item_id,self.userItem,self.tabelaSimilaridade,mediaUser,self.k,self.calculaSimilaridade)
                    y_pred.append(temp)
                elif self.calculaSimilaridade=='pearson':
                    temp=user_based(user_id,item_id,self.userItem,self.tabelaSimilaridade,mediaUser,self.k,self.calculaSimilaridade)
                    y_pred.append(temp)
                else:
                    display('Métrica de similaridade não encontrada')
                # self.userItem[user_id-1][item_id-1]=temp
                # mediaUser=self.userItem.mean(axis=1)
        return y_pred
        
    def evaluate(self, X, y):
        y_pred = self.predict(X)
        return mean_squared_error(y, y_pred)

In [ ]:
# class ModelBased():
#     def __init__(self, num_factors=10, learning_rate=0.01, reg_param=0.02, num_epochs=20, metric='regularized SVD') :
#         self.num_factors = num_factors
#         self.learning_rate = learning_rate
#         self.reg_param = reg_param
#         self.num_epochs = num_epochs
#         self.metric = metric
#     def fit(self,ratings):
        

In [65]:
total=dataOriginal.drop(columns='timestamp')
total=total.sort_values(by='user id')
train,test=train_test_split(total,train_size=0.9,shuffle=True)
x_train=train.drop('rating',axis=1)
y_train=train['rating']
x_test=test.drop('rating',axis=1)
y_test=test['rating']
x_test

,user id,item id
87992,159,358
92346,783,895
64183,75,988
25983,454,642
21242,126,272
...,...,...
84684,922,83
14965,222,71
39913,474,238
94840,627,684


In [66]:
x_test=test.drop('rating',axis=1)
y_test=test['rating']
x_test

,user id,item id
87992,159,358
92346,783,895
64183,75,988
25983,454,642
21242,126,272
...,...,...
84684,922,83
14965,222,71
39913,474,238
94840,627,684


In [67]:
modelo=MemoryBased(similaridade,userItem,k=30,metrica='user_based',calculaSimilaridade='cossine')
modelo.fit(x_train,y_train)

array([[5., 3., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [68]:
y_pred=modelo.predict(x_test)

0.3097502972651605

'Não tem usuários para recomendar'

'Não tem usuários para recomendar'

'Não tem usuários para recomendar'

'Não tem usuários para recomendar'

...

'Não tem usuários para recomendar'

'Não tem usuários para recomendar'

'Não tem usuários para recomendar'

'Não tem usuários para recomendar'

'Não tem usuários para recomendar'

In [69]:
display(y_pred,y_test)


[2.5499729269283082,
 3.380081329708715,
 2.4470052628740526,
 3.191337947486945,
 4.6114855678738955,
 4.381913745696833,
 3.3751612367033648,
 3.8332422598822697,
 2.502952880138843,
 4.159529569469986,
 2.458986800648126,
 2.853628748515309,
 3.9567577526890925,
 3.1141228437785253,
 2.995125424405959,
 3.480470257645207,
 4.297733689604175,
 2.834181523108045,
 3.6073767141941273,
 3.4894431613528316,
 3.7930599125909046,
 3.4413316737075723,
 2.951072524512786,
 2.5724653359472867,
 2.984244736026831,
 3.371410599251375,
 3.9416102567793674,
 1.1410530688292233,
 3.6676538137387444,
 3.3818215032550727,
 4.381395512528444,
 2.9653609804312633,
 2.682924335481904,
 4.602007071961381,
 3.553314574228187,
 3.8406582700678364,
 4.184660275791545,
 3.5145997624600103,
 3.6371090855899846,
 4.08718659474923,
 3.4584202687525365,
 3.004307682707379,
 3.309568780310031,
 3.7962201871778047,
 3.768009012696121,
 3.7310702921355157,
 3.5176770684129335,
 3.791497005228745,
 4.19468142381303

87992    1
92346    4
64183    2
25983    2
21242    3
        ..
84684    4
14965    4
39913    4
94840    4
32131    1
Name: rating, Length: 10000, dtype: int64

In [70]:
diff = abs(y_pred - y_test)
print(diff.sort_values(ascending=False).head(4000))


3774     4.589774
43688    4.482759
26400    3.987515
87956    3.839477
9542     3.804845
           ...   
32582    0.879366
73296    0.879321
16546    0.879312
79653    0.879278
98306    0.878865
Name: rating, Length: 4000, dtype: float64


In [71]:
mean_absolute_error(y_test,y_pred)

0.8238096153857967

In [72]:
np.sqrt(mean_squared_error(y_test,y_pred))


1.0284741585619381